# Computation of Granularity Adjustments of MDB Portfolios with a multi-factor model

In [1]:
import numpy as np
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt 
import pandas as pd
import copy
from tqdm import tqdm
%load_ext autoreload
%autoreload 2
%aimport IS_misc_functions
from IS_misc_functions import *
from misc_functions import *
from actuarial_training import *
from MtM_training import *
from scipy.optimize import minimize

import warnings
warnings.filterwarnings("ignore")

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import regularizers
# from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler,normalize
# from sklearn.model_selection import train_test_split
# from sklearn.utils import shuffle

# Create Portfolios

Read in everything

In [2]:
by_country = pd.read_excel("MDB portfolios.xlsx", sheet_name='by country')
maturity = pd.read_excel("MDB portfolios.xlsx", sheet_name='maturity')
summary = pd.read_excel("MDB portfolios.xlsx", sheet_name='summary of current info')
comments = pd.read_excel("MDB portfolios.xlsx", sheet_name='comments')
rating = pd.read_excel("MDB portfolios.xlsx", sheet_name='rating')
transition_matrix_SP = pd.read_csv("transition_matrix_SP.csv",delimiter = ",",index_col = 0)
transition_matrix_RC = pd.read_csv("transition_matrix_RC.csv",delimiter = ";",index_col = 0)

Preprocessing the Data

### Display the S & P Transition Matrix

In [3]:
SP_dict = trans_matrix_to_dict(transition_matrix_SP)
transition_matrix_SP

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A,0.00,0.00,0.00,1.15,12.33,77.29,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.01
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.82,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.02
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.39,70.86,11.24,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.04
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.60,68.05,11.16,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.06
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.94,74.69,6.50,2.13,0.27,0.08,0.15,0.12,0.08,0.11


### Display the PCT-adjusted  S & P Transition Matrix

In [4]:
RC_dict = trans_matrix_to_dict(transition_matrix_RC)
transition_matrix_RC

,AAA,AA+,AA,AA-,A+,A,A-,BBB+,BBB,BBB-,BB+,BB,BB-,B+,B,B-,Cs,D
AAA,96.79,2.71,0.42,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,0.00
AA+,6.45,85.16,6.61,1.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AA,0.00,6.22,85.17,6.74,0.52,0.42,0.10,0.52,0.00,0.00,0.00,0.31,0.00,0.00,0.00,0.00,0.00,0.00
AA-,0.00,0.00,7.82,83.45,7.16,0.17,0.50,0.17,0.00,0.17,0.44,0.00,0.00,0.11,0.00,0.00,0.00,0.00
A+,0.00,0.00,0.07,13.35,73.28,9.30,2.03,1.12,0.14,0.63,0.07,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A,0.00,0.00,0.00,1.15,12.33,77.30,5.71,1.68,0.77,0.96,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00
A-,0.00,0.00,0.00,0.00,0.94,11.47,77.83,6.94,0.41,1.57,0.67,0.16,0.00,0.00,0.00,0.00,0.00,0.01
BBB+,0.00,0.00,0.00,0.00,0.00,2.16,12.40,70.87,11.25,2.41,0.60,0.24,0.06,0.00,0.00,0.00,0.00,0.01
BBB,0.00,0.00,0.00,0.00,0.00,0.00,1.87,16.61,68.08,11.17,0.99,0.11,0.00,0.50,0.22,0.11,0.33,0.02
BBB-,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.93,14.95,74.75,6.51,2.13,0.27,0.08,0.15,0.12,0.08,0.03


Create Dictionaries to assign PDs to the considered countries

In [5]:
#Small Tolerance to avoid 0s
tolerance = 10e-10

# list of banks
bank_names = list(summary.iloc[:,0])

# Exclude EIB and NIB from being considered
bank_names = [b for b in bank_names if b != "EIB" and b!= "NIB"]


# SP Rating of Countries
country_sp = {rating["Country"][i].lower().replace(" ", ""): 
              str(rating["sp_cb"][i]).replace(" ", "")  for i in range(len(rating["sp_cb"]))}

def create_PD_dict(transition_matrix,unrated_b_minus = True):
    country_dict = country_sp #copy.copy(country_sp)

    # The PDs of Rating Classes
    Rating_PD = {transition_matrix.index[i]: np.maximum(transition_matrix["D"][i],tolerance)  for i in range(len(transition_matrix.index))}

    # Companies with D rating are not in default wrt other MDBs:
    Rating_PD["D"] = Rating_PD["Cs"]
    Rating_PD["SD"] = Rating_PD["Cs"]
    
    if unrated_b_minus:
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC","CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"]
            if country_rating in ['0',0]:
                Rating_PD[country_rating] = Rating_PD["B-"] 

#         for country_rating in set(country_dict.values()):
#             if country_rating not in Rating_PD.keys():
#                 Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC","CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"
            if country_dict[country] in ['0',0]:
                country_dict[country] = "B-"
    
    elif unrated_b_minus == False:

        # Adjust the PDs further 
        for country_rating in set(country_dict.values()):
            if country_rating in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                Rating_PD[country_rating] = Rating_PD["Cs"] 

        for country_rating in set(country_dict.values()):
            if country_rating not in Rating_PD.keys():
                Rating_PD[country_rating] = "nan"

        for country in country_dict:
            if country_dict[country] in ["C","CC",'0',0,"CCC-","CCC","CCC+"]:
                country_dict[country] = "Cs"

    # Assign the ratings to the countries
    countries_PD = {country.lower().replace(" ", ""): Rating_PD[country_dict[country]] for country in country_dict.keys()}
    return countries_PD

In [6]:
countries_PD_SP = create_PD_dict(transition_matrix_SP)
countries_PD_RC = create_PD_dict(transition_matrix_RC)

### Diplay the used PDs

In [7]:
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(pd.DataFrame([countries_PD_SP,countries_PD_RC],index =["S&P PDs","RC PDs"]).transpose())

,S&P PDs,RC PDs
afghanistan,51.47,14.70
albania,1.46,0.42
algeria,0.90,0.26
andorra,0.04,0.01
angola,7.59,2.17
anguilla,7.59,2.17
antiguaandbarbuda,51.47,14.70
argentina,51.47,14.70
armenia,0.90,0.26
aruba,0.06,0.02


Find the spaces between banks (in the excel sheet)

In [8]:
spaces = np.where(pd.isnull(by_country.iloc[:,3]))[0]
spaces[0] = 1
spaces = np.append(spaces,len(by_country.iloc[:,3]))

# Correlations with MSCI and VIX

In [9]:
VIX_NLR = [-0.512969464,-0.445965784,-0.578281944,0.612735824]
SP_NLR = [0.624029387,0.68891061,0.585351943,0.139517357]
correlations_VIX = {"Africa":VIX_NLR[0], "Asia": VIX_NLR[1], "Europe_Middle_East":VIX_NLR[2],"Latin_America": VIX_NLR[3] }
correlations_SP = {"Africa":SP_NLR[0], "Asia": SP_NLR[1], "Europe_Middle_East":SP_NLR[2],"Latin_America": SP_NLR[3] }

# Cauchy distribution Parameters

In [10]:
VIX_Cauchy = [-0.12775656254175363, 0.5678324440594043]
SP_Cauchy = [0.09708657166318732, 0.42861864106922676]

### Create the portfolios

Define a function to create the portfolios

In [11]:
def create_portfolio(q = 0.99, ELGD_val = 0.45,
                     transition_matrix = transition_matrix_SP,
                     countries_PD = countries_PD_SP,
                    constant_rho = False,
                    rho = 0.35,
                    normalize_shares = True):
    portfolios = {}
    for i in range(len(bank_names)):
        considered_countries = np.array(by_country.iloc[:,1][(spaces[i]+1):(spaces[i+1]-1)])
        PD = []
        EAD = []
        g = []
        rho_vix = []
        rho_sp = []
        for j in range(len(considered_countries)):
            country = considered_countries[j].lower().replace(" ", "")
            if country in countries_PD.keys() and countries_PD[country] != 'nan': # nur anlegen wenn existent
                PD.append(countries_PD[country] /100)
                EAD.append(by_country.iloc[:,3][(spaces[i]+1)+j])
                g.append(S-list(transition_matrix.index).index(country_sp[country]))
                rho_vix.append(correlations_VIX[by_country.iloc[:,7][(spaces[i]+1)+j]])
                rho_sp.append(correlations_SP[by_country.iloc[:,7][(spaces[i]+1)+j]])
        if normalize_shares:
            EAD = EAD/np.sum(np.array(EAD,dtype = np.float64))
        PD = np.array(PD)
        ELGD = np.array([ELGD_val]*len(EAD))
        if constant_rho:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = rho
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        else:
            def to_solve(om):
                b = (0.11852-0.05478*np.log(PD))**2
                R_corr = 0.12*(1-np.exp(-50*PD))/(1-np.exp(-50))+0.24*(1-(1-np.exp(-50*PD))/(1-np.exp(-50)))
                K = ((ELGD)*norm.cdf(np.sqrt(1/(1-R_corr))*norm.ppf(PD)+np.sqrt(R_corr/(1-R_corr))*norm.ppf(q))-PD*ELGD)
                alpha_X =  gamma.ppf(q,a =0.25, scale = 1/0.25)
                return  (np.abs(ELGD*PD*om*(alpha_X-1)-K))
        omega = fsolve(to_solve,[0.25]*len(EAD))
        omega = np.minimum(omega,1)

        #bnds = [(0,1) for i in range(len(EAD))]
        #omega = minimize(to_solve,[1]*len(EAD), bounds=bnds).x
        portfolios[bank_names[i]] = [EAD,ELGD,PD,omega,g,rho_vix,rho_sp]
      
    return portfolios

Create the portfolios for different quantile levels and different transiiton matricies under different assumptions

In [12]:
portfolios_099 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP)
portfolios_0999 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP)
portfolios_099_RC = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC)
portfolios_0999_RC = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC)


# Create Portfolios for rho = 0.12
portfolios_099_rho012 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.12)
portfolios_0999_rho012 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.12)
portfolios_099_RC_rho012 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.12)
portfolios_0999_RC_rho012 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.12)

# Create Portfolios for rho = 0.24
portfolios_099_rho024 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.24)
portfolios_0999_rho024 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.24)
portfolios_099_RC_rho024 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.24)
portfolios_0999_RC_rho024 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.24)

# Create Portfolios for rho = 0.35
portfolios_099_rho035 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.35)
portfolios_0999_rho035 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,
                                        constant_rho = True,rho = 0.35)
portfolios_099_RC_rho035 = create_portfolio(q = 0.99,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.35)
portfolios_0999_RC_rho035 = create_portfolio(q = 0.999,transition_matrix = transition_matrix_RC,countries_PD = countries_PD_RC,
                                        constant_rho = True,rho = 0.35)

PD Distribution

In [13]:
all_PDs = np.unique([v for v in countries_PD_SP.values()])
all_PDs_dict = {}
for PD in all_PDs:
    all_PDs_dict[np.round(PD,2)] = 0
portfolios_099_plot = create_portfolio(q = 0.99,transition_matrix = transition_matrix_SP,countries_PD = countries_PD_SP,normalize_shares = False)
Porfolio_PDs_list = []
for pf in portfolios_099_plot:
    pf_PDs = np.unique([v for v in countries_PD_SP.values()])
    pf_PDs_dict = {}
    for PD in all_PDs:
        pf_PDs_dict[np.round(PD,2)] = 0
    EAD = portfolios_099_plot[pf][0]
    PD = portfolios_099_plot[pf][2]
    for i in range(len(EAD)):        
        pf_PDs_dict[np.round(100*PD[i],2)] += EAD[i]
    pf_exposures = np.sum([v for v in pf_PDs_dict.values()])    
    for k in all_PDs_dict.keys():
        pf_PDs_dict[k] = pf_PDs_dict[k]/pf_exposures
    Porfolio_PDs_list.append(pf_PDs_dict)
PD_mean = {}
for PD in all_PDs:
    PD_mean[np.round(PD,2)] = np.mean([Porfolio_PDs_list[i][np.round(PD,2)] for i in range(len(Porfolio_PDs_list))])
PD_mean

{0.0: 0.0004962371935637645,
 0.01: 0.022976213971695594,
 0.02: 0.008815027375698609,
 0.04: 0.02629989043190439,
 0.06: 0.06454456923775069,
 0.11: 0.0586540776081741,
 0.18: 0.06928049192349311,
 0.4: 0.031108566905036324,
 0.9: 0.11070443733420475,
 1.46: 0.07672639359254495,
 2.38: 0.1992252347336631,
 7.59: 0.10282269901678953,
 51.47: 0.22834616067548105}

In [14]:
import json
print("Started writing dictionary to a file")
with open("probs.txt", "w") as fp:
    json.dump(PD_mean, fp)  # encode dict into JSON
print("Done writing dict into .txt file")

Started writing dictionary to a file
Done writing dict into .txt file


# Test everything

Define a Function for the computations of the granularity adjustment (GA)

In [15]:
def compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_099,
               trans_dict = SP_dict,
              n_mc = 5000000, 
              maturity = np.array([1]*len(portfolios_099)),
              LGD_constant = True):
    GA_MC_IRB = []
    GA_MC_MF = []
    GA_MC_com = []    
    GA_MC_com2 = []  
    GA_MC_t = []
    GA_MC_t2 = []
    GA_MC_t3 = []
    GA_MC_IRB_rel = []
    GA_MC_com_rel = []  
    GA_MC_com2_rel = []  
    GA_MC_MF_rel = []
    GA_MC_t_rel = []
    GA_MC_t2_rel = []
    GA_MC_t3_rel = []
    i = -1
    for portfolio in portfolios:
        i = i +1
        EAD,ELGD,PD,omega,g,rho_vix,rho_sp = portfolios[portfolio]
        ELGD = np.array([ELGD_const]*len(EAD))
        if len(EAD)>100: # Choose 100 indices
            indices = np.random.choice(range(len(EAD)-1), 100,replace = False)
            EAD = EAD[indices]
            PD = PD[indices]
            omega = omega[indices]
            ELGD = ELGD[indices]
            g = np.array(g)[indices]
        EAD = EAD/np.sum(EAD)   
        RHO = np.reshape([rho_vix,rho_sp],(1,2,len(PD)))
        
        GA,cond_loss = GA_IRB_MC2(PD, # Vector of default probabillities
          ELGD, #Vector of expected losses given default,
          EAD,  # EAD
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1)        
        
        GA_MF,cond_loss_MF = GA_IRB_MC_twofactor(PD, # Vector of default probabillities
          ELGD, #Vector of expected losses given default,
          EAD, # Vector of Exposures at default,
          RHO,  # Vector of Correlations
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1)
        
        GA_com,cond_loss_com = GA_comparable_one_factor(PD, # Vector of default probabillities
          ELGD, #Vector of expected losses given default,
          EAD, # Vector of Exposures at default,
          RHO,  # Vector of Correlations
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1)
        
        GA_com2,cond_loss_com2 = GA_IRB_MC2(PD, # Vector of default probabillities
          ELGD, #Vector of expected losses given default,
          EAD,  # EAD
          q = 0.999, # quantile level
          N_sim = n_mc,
          rho_input = True,
          rho_input_val = np.sum(RHO**2,axis = 1)**2,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1)     
        
        GA_t,cond_loss_t = GA_IRB_MC_t_distribution(PD, # Vector of default probabillities
        ELGD, #Vector of expected losses given default,
          EAD, # Vector of Exposures at default,
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1,
          df = 5)  

        GA_t2,cond_loss_t2 = GA_IRB_MC_t_distribution(PD, # Vector of default probabillities
        ELGD, #Vector of expected losses given default,
          EAD, # Vector of Exposures at default,
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1,
          df = 10)  
        
        GA_t3,cond_loss_t3 = GA_IRB_MC_t_distribution(PD, # Vector of default probabillities
        ELGD, #Vector of expected losses given default,
          EAD, # Vector of Exposures at default,
          q = 0.999, # quantile level
          N_sim = n_mc,
          nu = 0.25,
          LGD_constant = LGD_constant,
          M = 1,
          df = 50)  
        
        K_star = cond_loss - np.sum(PD*ELGD*EAD)
        GA_MC_IRB_rel.append(GA/(GA+K_star))              
        K_star_MF = cond_loss_MF - np.sum(PD*ELGD*EAD)
        GA_MC_MF_rel.append(GA_MF/(GA_MF+K_star_MF))
        K_star_com = cond_loss_com - np.sum(PD*ELGD*EAD)
        GA_MC_com_rel.append(GA_com/(GA_com+K_star_com))   
        K_star_com2 = cond_loss_com2 - np.sum(PD*ELGD*EAD)
        GA_MC_com2_rel.append(GA_com2/(GA_com2+K_star_com2))   
        K_star_t = cond_loss_t - np.sum(PD*ELGD*EAD)
        GA_MC_t_rel.append(GA_t/(GA_t+K_star_t))
        K_star_t2 = cond_loss_t2 - np.sum(PD*ELGD*EAD)
        GA_MC_t2_rel.append(GA_t2/(GA_t2+K_star_t2))     
        K_star_t3 = cond_loss_t3 - np.sum(PD*ELGD*EAD)
        GA_MC_t3_rel.append(GA_t3/(GA_t3+K_star_t3))       
        
        GA_MC_IRB.append(GA)
        GA_MC_MF.append(GA_MF)  
        GA_MC_t.append(GA_t)
        GA_MC_t2.append(GA_t2)
        GA_MC_t3.append(GA_t3)
        GA_MC_com.append(GA_com)
        GA_MC_com2.append(GA_com2)
        df = pd.DataFrame([[len(portfolios[portfolio][0])/100 for portfolio in portfolios],
                                            GA_MC_IRB,GA_MC_MF,GA_MC_com,GA_MC_com2,GA_MC_t,GA_MC_t2,GA_MC_t3,
                                          GA_MC_IRB_rel,GA_MC_MF_rel,GA_MC_com_rel,GA_MC_com2_rel,GA_MC_t_rel,GA_MC_t2_rel,GA_MC_t3_rel], 
                             columns=bank_names, 
                             index = ["Borrowers",
                                      "GA MC Single Factor, Normal distribution, IRB rho",
                                      "GA MC Multi Factor, Normal distribution",
                                      "GA comparable one factor model Pykhtin, Normal distribution",
                                      "GA comparable one factor model same factor, Normal distribution",
                                     "GA MC Single Factor, t Distribution, df = 5",
                                     "GA MC Single Factor, t Distribution, df = 10",
                                     "GA MC Single Factor, t Distribution, df = 50",
                                      "GA MC Single Factor, Normal distribution, IRB rho, Relative to UL",
                                      "GA MC Multi Factor, Normal distribution, Relative to UL",
                                     "GA comparable one factor model Pykhtin,  Relative to UL",
                                     "GA comparable one factor model same factor,  Relative to UL",
                                     "GA MC Single Factor, t Distribution, df =5, Relative to UL",
                                    "GA MC Single Factor, t Distribution, df =10, Relative to UL",
                                    "GA MC Single Factor, t Distribution, df =50, Relative to UL"])
    return df*100 # Output everything in percent

# GA Computations for S&P Transition Matrix

In [16]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 1.) q = 99.9%, ELGD = 0.45

In [17]:
df_999 = compute_GA(q=0.999,ELGD_const = 0.45,
               portfolios = portfolios_0999)
display(df_999)

,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
"GA MC Single Factor, Normal distribution, IRB rho",7.29,4.48,5.32,5.97,8.96,11.82,25.19,2.83,6.08,9.94,5.67
"GA MC Multi Factor, Normal distribution",4.06,2.05,1.94,3.71,4.52,8.65,8.70,1.00,2.45,3.92,1.49
"GA comparable one factor model Pykhtin, Normal distribution",4.13,1.97,2.04,3.79,4.61,8.64,8.79,1.75,2.49,3.89,1.57
"GA comparable one factor model same factor, Normal distribution",6.72,2.80,3.15,5.96,8.87,11.93,13.02,1.81,3.15,7.59,3.10
"GA MC Single Factor, t Distribution, df = 5",-2.62,-3.22,-5.17,-3.44,-2.54,-1.10,7.09,-4.61,-1.76,-0.77,-4.23
"GA MC Single Factor, t Distribution, df = 10",3.46,1.94,1.52,2.68,4.85,7.47,19.15,0.38,2.91,5.57,2.31
"GA MC Single Factor, t Distribution, df = 50",6.70,4.14,4.71,5.54,8.23,11.16,24.32,2.46,5.52,9.25,5.22
"GA MC Single Factor, Normal distribution, IRB rho, Relative to UL",46.58,47.00,38.14,46.84,45.89,57.32,75.72,36.26,32.04,48.22,47.77
"GA MC Multi Factor, Normal distribution, Relative to UL",20.61,12.59,7.13,22.84,16.91,34.96,19.79,8.22,8.84,12.70,6.44


## 2.) q = 99.9 %, ELGD = 0.1

In [18]:
df_999_lgd01 = compute_GA(q=0.999,ELGD_const = 0.1,
               portfolios = portfolios_0999)
display(df_999_lgd01)

,CAF,ADB,AFDB,IDB,CDB,CABEI,EADB,IBRD,TDB,BOAD,EBRD
Borrowers,16.00,38.00,29.00,26.00,16.00,11.00,4.00,78.00,21.00,8.00,37.00
"GA MC Single Factor, Normal distribution, IRB rho",1.62,1.00,1.15,1.33,1.99,2.63,5.60,0.62,1.35,2.21,1.27
"GA MC Multi Factor, Normal distribution",0.89,0.47,0.42,0.83,1.00,1.92,1.97,0.22,0.54,0.87,0.40
"GA comparable one factor model Pykhtin, Normal distribution",0.89,0.45,0.42,0.83,1.03,1.92,1.95,0.39,0.55,0.86,0.35
"GA comparable one factor model same factor, Normal distribution",1.49,0.61,0.69,1.32,1.97,2.65,2.89,0.41,0.69,1.69,0.69
"GA MC Single Factor, t Distribution, df = 5",-0.58,-0.71,-1.14,-0.76,-0.56,-0.25,1.58,-1.01,-0.39,-0.17,-0.95
"GA MC Single Factor, t Distribution, df = 10",0.77,0.42,0.37,0.59,1.01,1.65,4.26,0.08,0.64,1.24,0.51
"GA MC Single Factor, t Distribution, df = 50",1.49,0.93,1.05,1.21,1.83,2.48,5.40,0.54,1.22,2.05,1.16
"GA MC Single Factor, Normal distribution, IRB rho, Relative to UL",46.58,47.06,37.41,46.84,45.89,57.32,75.72,36.05,32.04,48.22,47.87
"GA MC Multi Factor, Normal distribution, Relative to UL",20.23,12.86,6.95,23.02,16.98,35.01,20.14,8.22,8.79,12.60,7.72
